In [1]:
!pip install gradio openai gtts pydub numpy requests groq openai-whisper
!apt-get install -y ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 14.4 MB/s eta 0:00:00
  Installing build dependencies ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.

In [25]:
import os
os.environ["GROQ_API_KEY"] = ""

In [30]:
import os
import gradio as gr
import whisper
from gtts import gTTS
import io
from groq import Groq

# Initialize the Groq client
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Load the Whisper model
model = whisper.load_model("base")  # You can choose other models like "small", "medium", "large"

def process_audio(file_path):
    try:
        if file_path is None:
            return "No file uploaded. Please upload an audio file.", None

        # Load the audio file
        audio = whisper.load_audio(file_path)

        # Transcribe the audio using Whisper
        result = model.transcribe(audio)
        text = result["text"]

        # Generate a response using Groq
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": text}],
            model="llama3-8b-8192",  # Replace with the correct model if necessary
        )

        # Access the response using dot notation
        response_message = chat_completion.choices[0].message.content.strip()

        # Convert the response text to speech
        tts = gTTS(response_message)
        response_audio_io = io.BytesIO()
        tts.write_to_fp(response_audio_io)  # Save the audio to the BytesIO object
        response_audio_io.seek(0)

        # Save audio to a file to ensure it's generated correctly
        with open("response.mp3", "wb") as audio_file:
            audio_file.write(response_audio_io.getvalue())

        # Return the response text and the path to the saved audio file
        return response_message, "response.mp3"

    except Exception as e:
        return f"An error occurred: {e}", None

# Define the interface with input and output sections
def chatbot_ui():
    with gr.Blocks(css="""
        body {
            background: linear-gradient(135deg, #FFB6C1, #FFC0CB, #FF69B4, #FF1493) !important;
            color: #000000;
        }
        .gradio-container {
            background: linear-gradient(135deg, #FFB6C1, #FFC0CB, #FF69B4, #FF1493) !important;
            color: #000000;
        }
        h3 {color: #1E90FF;} /* DodgerBlue color for name */
        h2 {color: #FF6347;} /* Tomato color for headers */
        .input-container, .output-container {
            background: rgba(255, 255, 255, 0.8) !important; /* Slightly transparent white background for inputs and outputs */
            color: #000000;
        }
    """) as iface:
        with gr.Row():
            with gr.Column(scale=2):
                gr.HTML("<h3>Created by: <a href='https://www.linkedin.com/in/datascientisthameshraj/' target='_blank' style='color: #1E90FF;'>Engr. Hamesh Raj</a></h3>")
                gr.HTML("<h2>🗣️ Chatbot Interface</h2>")

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 🎙️ Audio Input")
                audio_input = gr.Audio(type="filepath", label="Upload your audio file here")
                submit_btn = gr.Button("Submit")

            with gr.Column(scale=1):
                gr.Markdown("### 🔊 Response")
                response_text = gr.Textbox(label="Response Text")
                response_audio = gr.Audio(label="Response Audio")

        submit_btn.click(process_audio, inputs=audio_input, outputs=[response_text, response_audio])

    iface.launch()

# Launch the Gradio interface
chatbot_ui()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3e180f171a67076b30.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
